# VacationPy
---

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from config import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nagato,34.3833,131.2000,0.49,60,100,6.34,JP,1674613185
1,1,deyskoye,43.4755,44.1618,0.08,92,12,1.23,RU,1674613431
2,2,hay river,60.8156,-115.7999,-14.01,85,100,1.54,CA,1674613431
3,3,atuona,-9.8000,-139.0333,25.86,78,6,6.95,PF,1674613172
4,4,mataura,-46.1927,168.8643,23.73,33,22,1.52,NZ,1674613373


---

### Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values

# My Criteria:
# A max temperature lower than 27 degrees but higher than 21
# Cloudiness less than 3
# Wind speed less than 4.5 m/s


weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,lazaro cardenas,17.9583,-102.2000,23.72,67,0,2.71,MX,1674613440
287,287,takoradi,4.8845,-1.7554,26.25,84,0,1.73,GH,1674613304
315,315,kidal,18.4411,1.4078,22.88,14,1,3.63,ML,1674613544
357,357,ikom,5.9667,8.7063,22.09,85,1,0.96,NG,1674613557
392,392,horizontina,-27.6258,-54.3078,23.21,39,1,0.95,BR,1674613305
429,429,tiquicheo,18.8833,-100.7333,24.51,27,1,0.88,MX,1674613585
540,540,puerto escondido,15.8500,-97.0667,26.46,73,0,0.00,MX,1674613391


### Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,lazaro cardenas,MX,17.9583,-102.2000,67,
287,takoradi,GH,4.8845,-1.7554,84,
315,kidal,ML,18.4411,1.4078,14,
357,ikom,NG,5.9667,8.7063,85,
392,horizontina,BR,-27.6258,-54.3078,39,
429,tiquicheo,MX,18.8833,-100.7333,27,
540,puerto escondido,MX,15.8500,-97.0667,73,


### For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
takoradi - nearest hotel: Hillcrest
kidal - nearest hotel: No hotel found
ikom - nearest hotel: Gold Palm Hotels
horizontina - nearest hotel: Ouro Verde
tiquicheo - nearest hotel: No hotel found
puerto escondido - nearest hotel: Hotel Barlovento


,City,Country,Lat,Lng,Humidity,Hotel Name
34,lazaro cardenas,MX,17.9583,-102.2000,67,Hotel Sol del Pacífico
287,takoradi,GH,4.8845,-1.7554,84,Hillcrest
315,kidal,ML,18.4411,1.4078,14,No hotel found
357,ikom,NG,5.9667,8.7063,85,Gold Palm Hotels
392,horizontina,BR,-27.6258,-54.3078,39,Ouro Verde
429,tiquicheo,MX,18.8833,-100.7333,27,No hotel found
540,puerto escondido,MX,15.8500,-97.0667,73,Hotel Barlovento


### Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)